In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In this part, a regression model was performed to predict whether a player subscribes to the news letter based on demographic and behavioral features. Using cleaned and merged data, we selected key predictors including age, total played hours, number of sessions, average session duration, and total play duration. Moreover, the dataset was split into training (70%) and testing (30%) sets. 

The results show that session-based features, especially playtime and engagement, are a valuable metric to distinguish subscribers from non-subscribers. Therefore, this model provides a solid baseline for understanding how player behavior correlates with subscription likelihood. 

In [4]:
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")

sessions <- sessions |>
  mutate(start_time = dmy_hm(start_time),
    end_time = dmy_hm(end_time),
    session_duration_minutes = as.numeric(difftime(end_time, start_time, units = "mins")))

session_summary <- sessions |>
  group_by(hashedEmail) |>
  summarise(total_sessions = n(),
    avg_session_duration = mean(session_duration_minutes, na.rm = TRUE),
    total_play_duration = sum(session_duration_minutes, na.rm = TRUE))

player_data <- players |>
  left_join(session_summary, by = "hashedEmail") |>
  mutate(total_sessions = replace_na(total_sessions, 0),
    avg_session_duration = replace_na(avg_session_duration, 0),
    total_play_duration = replace_na(total_play_duration, 0),
    Age = if_else(is.na(Age), median(Age, na.rm = TRUE), Age),
    subscribe = as.factor(subscribe),
    experience = as.factor(experience),
    gender = as.factor(gender))

model_data <- player_data |>
  select(subscribe, Age, played_hours, total_sessions, avg_session_duration, total_play_duration)

set.seed(123)
split <- createDataPartition(model_data$subscribe, p = 0.7, list = FALSE)
train_data <- model_data[split, ]
test_data <- model_data[-split, ]

model <- train(subscribe ~ .,
               data = train_data,
               method = "glm",
               family = "binomial",
               trControl = trainControl(method = "none"))

predictions <- predict(model, newdata = test_data)
confusionMatrix(predictions, test_data$subscribe)

results <- test_data |>
  select(subscribe) |>
  mutate(predicted = predictions)

head(results)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in createDataPartition(model_data$subscribe, p = 0.7, list = FALSE): could not find function "createDataPartition"
